<h1>260250932 - Jonathan Adrian - Recommendation System</h1>

Pertama, semua library yang diperlukan diimport terlebih dahulu. Setelah itu, dataset yang akan digunakan diload dan dieksplorasi untuk melihat contoh data, mengetahui tipe data setiap kolom, serta jumlah baris dan kolom dalam dataset

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv('tourism_with_id.csv')

In [3]:
df.head()

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Unnamed: 11,Unnamed: 12
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,4.6,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153,NaN,1
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,4.6,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125,NaN,2
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000,4.6,360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538,NaN,3
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000,4.5,NaN,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-6.302446,106.895156,NaN,4
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000,4.5,60.0,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,106.839134,NaN,5


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437 entries, 0 to 436
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Place_Id      437 non-null    int64  
 1   Place_Name    437 non-null    object 
 2   Description   437 non-null    object 
 3   Category      437 non-null    object 
 4   City          437 non-null    object 
 5   Price         437 non-null    int64  
 6   Rating        437 non-null    float64
 7   Time_Minutes  205 non-null    float64
 8   Coordinate    437 non-null    object 
 9   Lat           437 non-null    float64
 10  Long          437 non-null    float64
 11  Unnamed: 11   0 non-null      float64
 12  Unnamed: 12   437 non-null    int64  
dtypes: float64(5), int64(3), object(5)
memory usage: 44.5+ KB


In [5]:
df.shape

(437, 13)

- Kolom Time_Minutes dapat dihapus karena memiliki jumlah nilai null yang melebihi 50%. Jika missing value diimputasi, data menjadi tidak valid karena jumlah data asli lebih sedikit dibandingkan dengan data yang diimputasi
- Kolom Unnamed 11 dapat dihapus karena hanya terdapat data kosong
- Kolom Unnamed 12 dapat dihapus karena isinya sama dengan kolom Place_Id

In [6]:
df.drop(['Time_Minutes', 'Unnamed: 11', 'Unnamed: 12'], axis=1, inplace=True)

In [7]:
df.shape

(437, 10)

Dataset yang digunakan tidak displit agar cosine similarity dapat dihitung untuk semua tempat wisata dalam dataset

<h1>Cleansing</h1>

Pada tahap cleansing, semua huruf besar akan diubah menjadi huruf kecil, lalu simbol, tanda baca, dan whitespace yang tidak diperlukan akan dibersihkan. Setelah data dibersihkan, proses selanjutnya adalah mengubahnya ke dalam bentuk TF-IDF menggunakan library scikit-learn. Dalam proses ini, setiap kata dalam kolom Description akan dipecah menjadi fitur-fitur terpisah, sehingga jumlah kolom yang dihasilkan akan sesuai dengan jumlah kata unik yang terdapat dalam Description

In [8]:
def cleansing(df):
    df_clean=df.str.lower()
    df_clean=[re.sub(r"\d+","",i )for i in df_clean]
    df_clean=[re.sub(r'[^\w]', ' ', i)for i in df_clean]
    df_clean=[re.sub(r'\s+',' ',i)for i in df_clean]
    return df_clean

In [9]:
clean_text = cleansing(df['Description'])

In [10]:
vectorizer = TfidfVectorizer()
tfidf=vectorizer.fit_transform(clean_text)

In [11]:
tfidf_des=pd.DataFrame(tfidf.toarray(),columns=vectorizer.get_feature_names_out())
tfidf_des.head()

,abad,abah,abang,abdul,abdullah,abdurrahman,abraham,abrasi,abu,acara,...,ꦫꦗ,ꦫꦠ,ꦫꦥ,ꦫꦮ,ꦭꦩ,ꦭꦮ,ꦱꦔ,ꦱꦩꦱ,ꦱꦫ,ꦲꦢ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
tfidf_des.shape

(437, 6207)

Setelah proses TF-IDF, jumlah fitur yang dihasilkan menjadi sangat banyak. Untuk mengurangi fitur yang tidak diperlukan, dilakukan pembersihan stopword, yaitu kata-kata yang sering muncul tetapi tidak terlalu penting. Proses ini dilakukan menggunakan library nltk

In [13]:
col=tfidf_des.columns

nltk.download('stopwords')

list_stopwords = set(stopwords.words('indonesian'))

nonstop_tokens = [word for word in col if not word in list_stopwords]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jonat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
tfidf_des = tfidf_des[nonstop_tokens]

In [15]:
tfidf_des.columns[100:200]

Index(['amazing', 'ambarawa', 'amblasnya', 'amenitas', 'amerika', 'amfiteater',
       'amphitheater', 'amsterdam', 'an', 'anak', 'anaknya', 'ancaman',
       'ancol', 'and', 'andalan', 'andil', 'andir', 'andong', 'aneh', 'aneka',
       'anggap', 'anggapan', 'anggaran', 'anggo', 'anggota', 'anggrek',
       'anggreknya', 'anggun', 'angin', 'angka', 'angkasa', 'angkatan',
       'angke', 'angker', 'angklung', 'angkut', 'angkutan', 'angle',
       'anjungan', 'anjuran', 'anoa', 'anom', 'antasena', 'anteng', 'anti',
       'antik', 'antikarat', 'antila', 'antonius', 'antrian', 'antusias',
       'anyaman', 'anyar', 'apapun', 'apartement', 'api', 'apik',
       'aplikasikan', 'april', 'aquarium', 'aquarius', 'arab', 'arah',
       'arahan', 'arahkan', 'arca', 'arcamanik', 'architectuur', 'ardila',
       'ardilla', 'area', 'areal', 'arekan', 'arena', 'argentina', 'argo',
       'ari', 'arif', 'arifin', 'arkeolog', 'arkeologi', 'armada', 'arnowo',
       'aroma', 'arsip', 'arsitek', 'arsit

In [16]:
tfidf_des.head()

,abad,abah,abang,abdul,abdullah,abdurrahman,abraham,abrasi,abu,acara,...,ꦫꦗ,ꦫꦠ,ꦫꦥ,ꦫꦮ,ꦭꦩ,ꦭꦮ,ꦱꦔ,ꦱꦩꦱ,ꦱꦫ,ꦲꦢ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Sebelum menggabungkan fitur TF-IDF dengan fitur data lainnya, dilakukan reset index terlebih dahulu agar struktur data tetap rapi. Hal ini dilakukan karena proses concat akan membaca index, sehingga dengan reset index, data dapat digabungkan dengan kolom yang tepat

In [17]:
tfidf_des.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437 entries, 0 to 436
Columns: 5789 entries, abad to ꦲꦢ
dtypes: float64(5789)
memory usage: 19.3 MB


In [18]:
tfidf_des = tfidf_des.reset_index()
df = df.reset_index()

In [19]:
df_list = pd.concat([df,tfidf_des], axis=1)
df_list.head()

,index,Place_Id,Place_Name,Description,Category,City,Price,Rating,Coordinate,Lat,...,ꦫꦗ,ꦫꦠ,ꦫꦥ,ꦫꦮ,ꦭꦩ,ꦭꦮ,ꦱꦔ,ꦱꦩꦱ,ꦱꦫ,ꦲꦢ
0,0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,4.6,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,4.6,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000,4.6,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000,4.5,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-6.302446,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000,4.5,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Memeriksa apakah terdapat missing value dalam data

In [20]:
df_list.isnull().sum()

index          0
Place_Id       0
Place_Name     0
Description    0
Category       0
              ..
ꦭꦮ             0
ꦱꦔ             0
ꦱꦩꦱ            0
ꦱꦫ             0
ꦲꦢ             0
Length: 5801, dtype: int64

<h1>Encoding</h1>

Fitur kategorikal seperti Category dan City diubah menjadi bentuk numerik menggunakan One-Hot Encoding. Proses ini diperlukan karena model machine learning hanya dapat memproses data dalam bentuk numerik.

In [21]:
df_list['Category'].unique()

array(['Budaya', 'Taman Hiburan', 'Cagar Alam', 'Bahari',
       'Pusat Perbelanjaan', 'Tempat Ibadah'], dtype=object)

In [22]:
encoded=OneHotEncoder()
data=pd.DataFrame(encoded.fit_transform(df_list[['Category']]).toarray(),columns=encoded.get_feature_names_out())
df_list=pd.concat([df_list,data], axis=1)

In [23]:
df_list.head()

,index,Place_Id,Place_Name,Description,Category,City,Price,Rating,Coordinate,Lat,...,ꦱꦔ,ꦱꦩꦱ,ꦱꦫ,ꦲꦢ,Category_Bahari,Category_Budaya,Category_Cagar Alam,Category_Pusat Perbelanjaan,Category_Taman Hiburan,Category_Tempat Ibadah
0,0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,4.6,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,4.6,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000,4.6,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000,4.5,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-6.302446,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000,4.5,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [24]:
df_list['City'].unique()

array(['Jakarta', 'Yogyakarta', 'Bandung', 'Semarang', 'Surabaya'],
      dtype=object)

In [25]:
data=pd.DataFrame(encoded.fit_transform(df_list[['City']]).toarray(),columns=encoded.get_feature_names_out())
df_list=pd.concat([df_list,data], axis=1)

In [26]:
df_list.head()

,index,Place_Id,Place_Name,Description,Category,City,Price,Rating,Coordinate,Lat,...,Category_Budaya,Category_Cagar Alam,Category_Pusat Perbelanjaan,Category_Taman Hiburan,Category_Tempat Ibadah,City_Bandung,City_Jakarta,City_Semarang,City_Surabaya,City_Yogyakarta
0,0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,4.6,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,4.6,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000,4.6,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000,4.5,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-6.302446,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000,4.5,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


<h1>Scaling</h1>

Fitur numerik seperti Price dan Rating dinormalisasi menggunakan MinMaxScaler agar semua fitur memiliki skala yang sama, yaitu antara 0 dan 1. Hal ini penting untuk mencegah fitur dengan nilai besar, seperti Price, mendominasi perhitungan similarity

In [27]:
scaler = MinMaxScaler()
df_list['Price'] = scaler.fit_transform(df_list['Price'].values.reshape(-1, 1))

In [28]:
df_list.head()

,index,Place_Id,Place_Name,Description,Category,City,Price,Rating,Coordinate,Lat,...,Category_Budaya,Category_Cagar Alam,Category_Pusat Perbelanjaan,Category_Taman Hiburan,Category_Tempat Ibadah,City_Bandung,City_Jakarta,City_Semarang,City_Surabaya,City_Yogyakarta
0,0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,0.022222,4.6,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0.000000,4.6,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,0.300000,4.6,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,0.011111,4.5,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-6.302446,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,0.104444,4.5,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [29]:
df_list['Rating'] = scaler.fit_transform(df_list['Rating'].values.reshape(-1, 1))

In [30]:
df_list.head()

,index,Place_Id,Place_Name,Description,Category,City,Price,Rating,Coordinate,Lat,...,Category_Budaya,Category_Cagar Alam,Category_Pusat Perbelanjaan,Category_Taman Hiburan,Category_Tempat Ibadah,City_Bandung,City_Jakarta,City_Semarang,City_Surabaya,City_Yogyakarta
0,0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,0.022222,0.7500,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0.000000,0.7500,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,0.300000,0.7500,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,0.011111,0.6875,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-6.302446,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,0.104444,0.6875,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


<h1>Drop Column</h1>

Sebelum menghitung cosine similarity, perlu dipastikan bahwa hanya kolom yang bernilai numerik yang digunakan. Oleh karena itu, kolom dengan nilai kategorikal harus dihapus (drop) agar tidak memengaruhi perhitungan

In [31]:
df_list_drop = df_list.drop(['index','Place_Id','Place_Name','Description', 'Category', 'City', 'Coordinate'], axis=1)

In [32]:
df_list_drop.head()

,Price,Rating,Lat,Long,abad,abah,abang,abdul,abdullah,abdurrahman,...,Category_Budaya,Category_Cagar Alam,Category_Pusat Perbelanjaan,Category_Taman Hiburan,Category_Tempat Ibadah,City_Bandung,City_Jakarta,City_Semarang,City_Surabaya,City_Yogyakarta
0,0.022222,0.7500,-6.175392,106.827153,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.000000,0.7500,-6.137645,106.817125,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.300000,0.7500,-6.125312,106.833538,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.011111,0.6875,-6.302446,106.895156,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.104444,0.6875,-6.124190,106.839134,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


Perhitungan jarak similarity antara satu tempat wisata dengan tempat wisata lainnya dilakukan menggunakan persamaan Cosine Similarity, yang menghasilkan cosine matrix. Nilai cosine similarity yang semakin mendekati 1 menunjukkan bahwa dua tempat wisata tersebut memiliki tingkat kemiripan yang lebih tinggi

In [33]:
cos_sim = pd.DataFrame(cosine_similarity(df_list_drop))

In [34]:
cos_sim.head()

,0,1,2,3,4,5,6,7,8,9,...,427,428,429,430,431,432,433,434,435,436
0,1.000000,0.999924,0.999837,0.999838,0.999833,0.999835,0.999837,0.999826,0.997518,0.999826,...,0.999839,0.999826,0.999741,0.999742,0.999742,0.999813,0.999741,0.999741,0.999744,0.999737
1,0.999924,1.000000,0.999830,0.999835,0.999832,0.999834,0.999833,0.999824,0.997546,0.999825,...,0.999821,0.999823,0.999739,0.999738,0.999737,0.999812,0.999736,0.999735,0.999740,0.999738
2,0.999837,0.999830,1.000000,0.999925,0.999923,0.999954,0.999831,0.999916,0.997564,0.999827,...,0.999733,0.999733,0.999820,0.999821,0.999819,0.999720,0.999817,0.999814,0.999819,0.999729
3,0.999838,0.999835,0.999925,1.000000,0.999923,0.999929,0.999842,0.999915,0.997443,0.999826,...,0.999750,0.999749,0.999836,0.999839,0.999844,0.999738,0.999843,0.999832,0.999840,0.999747
4,0.999833,0.999832,0.999923,0.999923,1.000000,0.999931,0.999832,0.999918,0.997562,0.999826,...,0.999738,0.999735,0.999832,0.999819,0.999818,0.999721,0.999818,0.999821,0.999820,0.999731


Agar rekomendasi lebih mudah dipahami, Place_Name diubah menjadi index. Dengan cara ini, saat menampilkan rekomendasi, yang muncul adalah nama tempat wisata, bukan angka atau index numerik

In [35]:
df_list.set_index('Place_Name', inplace=True)

In [36]:
df_list.head()

,index,Place_Id,Description,Category,City,Price,Rating,Coordinate,Lat,Long,...,Category_Budaya,Category_Cagar Alam,Category_Pusat Perbelanjaan,Category_Taman Hiburan,Category_Tempat Ibadah,City_Bandung,City_Jakarta,City_Semarang,City_Surabaya,City_Yogyakarta
Place_Name,,,,,,,,,,,,,,,,,,,,,
Monumen Nasional,0,1,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,0.022222,0.7500,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Kota Tua,1,2,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0.000000,0.7500,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Dunia Fantasi,2,3,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,0.300000,0.7500,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
Taman Mini Indonesia Indah (TMII),3,4,Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,0.011111,0.6875,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-6.302446,106.895156,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
Atlantis Water Adventure,4,5,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,0.104444,0.6875,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,106.839134,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [37]:
df_list.shape

(437, 5811)

Mengambil index dari data dan menampilkan 10 index pertama

In [38]:
indices = pd.Series(df_list.index)

In [39]:
indices[:10]

0                     Monumen Nasional
1                             Kota Tua
2                        Dunia Fantasi
3    Taman Mini Indonesia Indah (TMII)
4             Atlantis Water Adventure
5              Taman Impian Jaya Ancol
6               Kebun Binatang Ragunan
7                        Ocean Ecopark
8                     Pelabuhan Marina
9                         Pulau Tidung
Name: Place_Name, dtype: object

Fungsi rekomendasi dibuat untuk memberikan rekomendasi tempat wisata. Pertama, nilai cosine similarity diambil berdasarkan index Place_Name, kemudian diurutkan dari yang terbesar ke terkecil. Setelah itu, ketika fungsi rekomendasi dipanggil, sistem akan menampilkan 5 rekomendasi tempat wisata berdasarkan content-based filtering model

In [40]:
def recommendations(title, cos_sim) :
    recommend_place = []
    
    idx = indices[indices==title].index[0]
    
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending=False)
    
    top_10_indexes = list(score_series.iloc[1:6].index)
    
    for i in top_10_indexes:
        recommend_place.append(list(df_list.index)[i])
        
    return recommend_place

In [41]:
recommendations('Caringin Tilu', cos_sim)

['Wisata Batu Kuda',
 'Bukit Moko',
 'Tafso Barn',
 'Sanghyang Heuleut',
 'Taman Bunga Cihideung']

<h1>Evaluation</h1>

Karena tidak ada baseline, evaluasi model dilakukan menggunakan precision@k dengan k = 5. Metode ini menghitung jumlah rekomendasi yang relevan (memiliki kategori yang sama) kemudian dibagi dengan total jumlah rekomendasi yang diberikan. Dalam kasus ini, diambil 10 sampel untuk dihitung nilai precision@k masing-masing. Setelah itu, seluruh nilai precision@k dijumlahkan dan dibagi dengan jumlah sampel untuk mendapatkan rata-rata precision@k dari 10 sampel tersebut

In [42]:
def precision_at_k(title, cos_sim, k=5):
    recommend_place = recommendations(title, cos_sim)
    
    input_category = df_list.loc[title, 'Category']
    
    relevant_count = sum(df_list.loc[place, 'Category'] == input_category for place in recommend_place)
    
    precision = relevant_count / k
    return precision

In [43]:
precision_at_k('Caringin Tilu', cos_sim, k=5)

1.0

In [44]:
def mean_precision_at_k(sample_titles, cos_sim, k=5):
    precisions = [precision_at_k(title, cos_sim, k) for title in sample_titles]
    
    return sum(precisions) / len(sample_titles)

Hasil evaluasi menampilkan rata-rata precision@k untuk 10 sampel tempat wisata

In [45]:
sample_titles = [
    'Caringin Tilu', 'Kebun Bibit Wonorejo', 'Setu Babakan', 'Taman Pintar Yogyakarta', 
    'Sanghyang Heuleut', 'Pasar Beringharjo', 'Desa Wisata Sungai Code Jogja Kota', 
    'Museum Sri Baduga', 'House of Sampoerna', 'Pantai Kukup'
]
mean_precision_score = mean_precision_at_k(sample_titles, cos_sim, k=5)
print(f'Mean Precision@5: {mean_precision_score:.2f}')

Mean Precision@5: 0.92
